In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import pickle
from skimage.filters import sobel, prewitt
from skimage.feature import local_binary_pattern as lbp, canny,corner_harris
import warnings
warnings.filterwarnings("ignore")
from time import sleep
import sys

In [2]:
X=pickle.load(open("data/X.pickle","rb"))

In [3]:
def ft_extractor(dataset):
    image_dataset=pd.DataFrame()
    for image in range(dataset.shape[0]):
        df=pd.DataFrame()
        input_image=dataset[image].reshape(96,96)
        img=input_image
        
        #pixel values
        pixel_values=img.reshape(-1)
        df["Pixel_Values"]=pixel_values
        
        #gabor filter
        num=1
        kernels=[]
        for theta in range(2):
            theta = theta/4.*np.pi
            for sigma in (1,3):
                lamda=np.pi/4
                gamma=0.5
                phi=0
                gabor_label="Gabor " +str(num)
                kernel_size=9
                kernel = cv2.getGaborKernel((kernel_size,kernel_size),sigma,theta,lamda,gamma,phi,ktype=cv2.CV_32F)
                kernels.append(kernel)
                fimg=cv2.filter2D(img,cv2.CV_8UC3,kernel)
                filtered_img=fimg.reshape(-1)
                df[gabor_label]=filtered_img
                num +=1
        
        #local binary patterns
        lbp_ft = lbp(img,8,1)
        ft_lbp = lbp_ft.reshape(-1)
        df['Local Binary Patterns'] = ft_lbp   
        
        #harris corner algorithm
        h_corner_ft=corner_harris(img)
        harris_ft=h_corner_ft.reshape(-1)
        df['Corner Features'] = harris_ft  
        
        #sobel edge filter
        sobel_ft = sobel(img)
        edge_s = sobel_ft.reshape(-1)
        df['Sobel'] = edge_s
    
        #prewitt edge filter
        prewitt_ft = prewitt(img)
        edge_p = prewitt_ft.reshape(-1)
        df['Prewitt'] = edge_p
        
        sys.stdout.write('\r')
        # the exact output you're looking for:
        sys.stdout.write("progress = " + str(round(image*100/len(dataset),2)) + "%")
        sys.stdout.flush()
        sleep(0.25)
        
        image_dataset=image_dataset.append(df)
    return image_dataset

In [4]:
X_extractedft=ft_extractor(X)

progress = 99.88%

In [5]:
X_extractedft

,Pixel_Values,Gabor 1,Gabor 2,Gabor 3,Gabor 4,Local Binary Patterns,Corner Features,Sobel,Prewitt
0,0,0,0,0,0,255.0,0.000000,0.000000,0.000000
1,0,0,0,0,0,255.0,0.000000,0.000000,0.000000
2,0,0,0,0,0,255.0,0.000000,0.000000,0.000000
3,0,0,0,0,0,255.0,0.000000,0.000000,0.000000
4,0,0,0,0,0,255.0,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
9211,255,146,126,255,255,17.0,-0.748006,0.707107,0.707107
9212,255,147,120,255,255,17.0,-0.712251,0.707107,0.707107
9213,255,138,41,255,255,17.0,-0.348416,0.707107,0.707107
9214,255,140,89,255,255,17.0,0.800816,0.680608,0.672026


In [8]:
image_features=np.expand_dims(X_extractedft, axis=0)
data_for_classification=np.reshape(image_features,(X.shape[0],-1))

In [10]:
data_for_classification=pickle.load(open("data/data_for_classification.pickle","rb"))
data_for_classification.shape

(849, 82944)

In [11]:
pickle_out=open("data/data_for_classification.pickle","wb")
pickle.dump(data_for_classification,pickle_out)
pickle_out.close()